# WA-1 (Raw Waveforms) Performance Testing

From WaveformDataFabric.pdf > COI Data Model > FDSN Operations > dataselect/1/query
> This operation has the following performance requirements:
>  1. The Data Fabric shall respond to an FDSN Waveform Data Select query requesting waveforms for up to 30 **Channel** objects, each 90 minutes duration at 40Hz sampling rate, in less than 6 seconds (goal: 3 seconds).
>  2. The Data Fabric shall simultaneously handle up to 75 FDSN Waveform Data Select queries (up to 30 **Channel** objects, 90 minutes duration, 40Hz sampling rate) without exceeding the response time requirement for any of the queries.


#### REPORT, 28 May 2025
Environment
- M1 Mac 
- remote connection to Sandia through VPN
- IEI version 1.8.5-1

Results:
1. Average response time: 19.38 seconds
2. Average response time: 54.26 seconds</br>
  Total response time: 235.21 seconds

##### Imports

In [1]:
import aiohttp
import asyncio
import json
import nest_asyncio
import os
import requests
import time

##### Variables users need to define

In [ ]:
# DAL FDSN Dataselect Service endpoint URL
service_url = "https://###.###.###.###/hydra/rest/waveform/dataselect/1/query"

# Location of file with credentials and username to lookup
connections_file = os.path.join(".", "connections.json")
user = "admin"

# Location of requests for tests
requests_location = os.path.join(".", "PerformanceRequests")

# Parameters space for testing - commentint out EPOCH and MSGPACK until supported
time_formats = ['ISO']#, 'EPOCH']
accepts = ['application/json']# , 'application/msgpack']

# Number of times to run each test - define for each test below until endpoint is performant
num_runs = 10

# Number of simulataneous requests to send - define for each test below until endpoint is performant
num_simul_requests = 75

##### Retrieve IEI authetication credentials from file

In [3]:
with open(connections_file, 'r') as file:
    connections_data = json.load(file)
    username = connections_data[user]["username"]
    password = connections_data[user]["password"]

### Performance Test #1
Request:
- 30 channels
- 90 minute duration
- 40 Hz sampling rate

Expected response:
- < 6 seconds


In [4]:
num_runs = 10

In [5]:
# For each response time format
for time_format in time_formats:

    # For each response file format
    for accept in accepts:

        # Print header info
        print(f"\n\n\n----- WA-1 PERFORMANCE TEST #1 / {time_format} / {accept} -----")

        # Declare appropriate headers
        headers = {
            'Accept': accept,
            'Content-Type': 'application/x-www-form-urlencoded',
            'Time-Format': time_format
        }

        # Instantiate total time for all requests
        total_time = 0

        # Read in request body from file
        with open(os.path.join(f"{requests_location}", f"wa-1_performance_test1_request_{time_format}.txt"), 'r') as file:
            request_body = file.read()

        # For each peformance test run
        for i in range(num_runs):

            # Hit the endpoint and calculate times + print
            start_time = time.time()
            
            response = requests.post(service_url, auth=(username, password), headers=headers, data=request_body)
            response_data = response.json()
            
            end_time = time.time()
            elapsed_time = end_time - start_time
            total_time += elapsed_time
            
            print(f"{i+1:02}/{num_runs:02} POST REQUEST returned in {elapsed_time:.2f} seconds. ")

            # Print status
            if response.status_code == 200:
                response_data = response.json()
                print(f'       Success: {response.status_code}, {len(response_data)} waveform segments returned')
            elif response.status_code == 204:
                print("        No waveform data returned.")
            else:
                print(f'       Failed: {response.status_code}, {response.reason}\n{response.text}')

        # Calculate average time of all runs
        average_time = total_time / num_runs

        # Print summary statistics
        print("\n----- SUMMARY -----")
        print(f"time-format: {time_format}")
        print(f"accepts: {accept}")
        print(f"Average return time: {average_time:.2f} seconds")
        print("-------------------------------------------------------------")




----- WA-1 PERFORMANCE TEST #1 / ISO / application/json -----
01/10 POST REQUEST returned in 19.41 seconds. 
       Success: 200, 570 waveform segments returned
02/10 POST REQUEST returned in 20.10 seconds. 
       Success: 200, 570 waveform segments returned
03/10 POST REQUEST returned in 19.25 seconds. 
       Success: 200, 570 waveform segments returned
04/10 POST REQUEST returned in 19.22 seconds. 
       Success: 200, 570 waveform segments returned
05/10 POST REQUEST returned in 19.31 seconds. 
       Success: 200, 570 waveform segments returned
06/10 POST REQUEST returned in 19.02 seconds. 
       Success: 200, 570 waveform segments returned
07/10 POST REQUEST returned in 19.32 seconds. 
       Success: 200, 570 waveform segments returned
08/10 POST REQUEST returned in 19.41 seconds. 
       Success: 200, 570 waveform segments returned
09/10 POST REQUEST returned in 19.41 seconds. 
       Success: 200, 570 waveform segments returned
10/10 POST REQUEST returned in 19.42 seconds

### Performance Test #2
Request:
- 30 channels
- 90 minute duration
- 40 Hz sampling rate
- 75 simulataneous requests

Expected response:
- each response < 6 seconds

In [6]:
num_runs = 1
num_simul_requests = 75

In [7]:
# Allow nested event loops
nest_asyncio.apply()

auth = aiohttp.BasicAuth(username, password)

async def fetch(session, service_url, request_body, headers):
    async with session.post(service_url, data=request_body, auth=auth, headers=headers) as response:
        start_time = time.time()
        response_data = await response.json()
        elapsed_time = time.time() - start_time
        return response_data, elapsed_time

async def main():

    response_times = []

    max_time=0
    sum_time=0
    sub_six=0

    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, service_url, request_body, headers) for _ in range(num_simul_requests)]
        results = await asyncio.gather(*tasks)

    # Extract response times from results
    for response_data, elapsed_time in results:
        response_times.append(elapsed_time)

    # Print the response times
    for i, elapsed_time in enumerate(response_times):
        if elapsed_time > max_time:
            max_time = elapsed_time
        sum_time += elapsed_time
        if elapsed_time < 6:
            sub_six += 1
        # print(f"Request {i + 1}: {elapsed_time:.2f} seconds")
    print(f"  Max Request Time: {max_time:.2f} seconds")
    print(f"  Mean Request Time: {sum_time/num_simul_requests:.2f} seconds")
    print(f"  Sub-6sec Returns: {sub_six} / {num_simul_requests} requests")

# For each response time format
for time_format in time_formats:

    # For each response file format
    for accept in accepts:

        # Print header info
        print(f"\n\n\n----- WA-1 PERFORMANCE TEST #2 / {time_format} / {accept} -----")

        # Declare appropriate headers
        headers = {
            'Accept': accept,
            'Content-Type': 'application/x-www-form-urlencoded',
            'Time-Format': time_format
        }

        # Read in request body from file
        with open(os.path.join(f"{requests_location}", f"wa-1_performance_test1_request_{time_format}.txt"), 'r') as file:
            request_body = file.read()

        overall_start_time = time.time()

        # For each performance test run
        for run_num in range(num_runs):

            # Print header
            print(f"\n  ----- Run #{run_num+1} -----")

            run_start_time = time.time()

            # Run the asynchronous, simulataneous requests
            asyncio.run(main())

            # Calcualte and print elapsed time
            run_elapsed_time = time.time() - run_start_time
            print(f"  Run #{run_num+1} elapsed time: {run_elapsed_time:.2f} seconds")
       
        # Calculate and print overall elapsed time
        overall_elapsed_time = time.time() - overall_start_time
        print(f"\nAll elapsed time: {overall_elapsed_time:.2f} seconds")
        print("-------------------------------------------------------------")




----- WA-1 PERFORMANCE TEST #2 / ISO / application/json -----

  ----- Run #1 -----
  Max Request Time: 78.18 seconds
  Mean Request Time: 54.26 seconds
  Sub-6sec Returns: 0 / 75 requests
  Run #1 elapsed time: 235.21 seconds

All elapsed time: 235.21 seconds
-------------------------------------------------------------
